In [19]:
import pandas as pd
from ipywidgets import interact, widgets
import plotly.express as px

# Read data from JSON file
df = pd.read_json('project_3.fridge_fills.json')

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: x['$date']))

# Drop '_id' column
df.drop('_id', axis=1, inplace=True, errors='ignore')

# Create a copy of the DataFrame with datetime 'Date' column
df_dates = df.copy()

# Define function to plot entries per month
def plot_entries_per_month(fill_type):
    if fill_type == 'All Fill Types':
        filtered_df = df_dates.copy()  
    else:
        filtered_df = df_dates[df_dates['Fill Type'] == fill_type].copy()  

    # Ensure datetime column has timezone information
    filtered_df['Date'] = filtered_df['Date'].dt.tz_localize(None)  # Remove any existing timezone information

    # Convert 'Date' column to Periods
    filtered_df.loc[:, 'YearMonth'] = filtered_df['Date'].dt.to_period('M').astype(str)

    monthly_counts = filtered_df.groupby('YearMonth').size().reset_index(name='Total Entries')

    # Set figure size
    fig = px.bar(monthly_counts, x='YearMonth', y='Total Entries', 
                 labels={'YearMonth': 'Month', 'Total Entries': 'Total Entries'},
                 title=f'Total Fills per Month for {fill_type}')
    fig.update_layout(xaxis_tickangle=-45, yaxis_gridcolor='lightgrey', 
                      width=800, height=500) 
    fig.show()

# Get unique values of 'Fill Type' for dropdown menu excluding 'nan'
fill_types = df_dates['Fill Type'].dropna().unique()
fill_types = list(fill_types)
fill_types.append('All Fill Types')  

# Create interactive dropdown menu
interact(plot_entries_per_month, fill_type=widgets.Dropdown(options=fill_types, description='Fill Type:'))


interactive(children=(Dropdown(description='Fill Type:', options=('Rescued Groceries', 'Purchased Groceries', …

<function __main__.plot_entries_per_month(fill_type)>